![baby_names](baby_names.jpg)


How have American baby name tastes changed since 1920? Which names have remained popular for over 100 years, and how do those names compare to more recent top baby names? These are considerations for many new parents, but the skills you'll practice while answering these queries are broadly applicable. After all, understanding trends and popularity is important for many businesses, too!

You'll be working with data provided by the United States Social Security Administration, which lists first names along with the number and sex of babies they were given to in each year. For processing speed purposes, the dataset is limited to first names which were given to over 5,000 American babies in a given year. The data spans 101 years, from 1920 through 2020.

## The Data

### `baby_names`

| column         | type    | description                                                                  |
| -------------- | ------- | ------------------------------------------------------------------------ |
| `year`         | int     | year                                                                     |
| `first_name`   | varchar | first name                                                               |
| `sex`          | varchar | `sex` of babies given `first_name`                                       |
| `num`          | int     | number of babies of `sex` given `first_name` in that `year`              |


In [11]:
-- Run this code to view the data in baby_names
SELECT *
FROM baby_names
LIMIT 5;

,year,first_name,sex,num
0,1920,Mary,F,70982
1,1920,Dorothy,F,36643
2,1920,Helen,F,35097
3,1920,Margaret,F,27994
4,1920,Ruth,F,26101


In [5]:

-- List the overall top five names in alphabetical order and find out if each name is "Classic" or "Trendy."
SELECT 
	first_name,
	SUM(num),
	CASE WHEN COUNT(first_name) > 10 THEN 'Trendy'
	ELSE 'Classic' END AS popularity_type
FROM baby_names
GROUP BY first_name
ORDER BY first_name ASC
LIMIT 5;


,first_name,sum,popularity_type
0,Aaliyah,15870,Classic
1,Aaron,530592,Trendy
2,Abigail,338485,Trendy
3,Adam,497293,Trendy
4,Addison,107433,Trendy


In [13]:

-- What were the top 20 male names overall, and how did the name Paul rank?
SELECT
  RANK() OVER (ORDER BY sum DESC) AS name_rank,
  first_name,
  sum
FROM (
  SELECT
    first_name,
    SUM(num) AS sum
  FROM
    baby_names
  GROUP BY
    first_name
) AS ranked_names
ORDER BY
  name_rank
LIMIT 20;

,name_rank,first_name,sum
0,1,James,4748138
1,2,John,4510721
2,3,Robert,4495199
3,4,Michael,4278824
4,5,William,3614424
5,6,David,3571498
6,7,Mary,3215850
7,8,Richard,2414838
8,9,Joseph,2361382
9,10,Thomas,2166802


In [14]:

-- Which female names appeared in both 1920 and 2020?

WITH names_1920 AS (
    SELECT first_name, num
    FROM baby_names
    WHERE year = 1920 AND sex = 'F'
),
names_2020 AS (
    SELECT first_name, num
    FROM baby_names
    WHERE year = 2020 AND sex = 'F'
)
SELECT 
    a.first_name,
    (a.num + b.num) AS total_occurrences
FROM names_1920 a
JOIN names_2020 b
    ON a.first_name = b.first_name;

,first_name,total_occurrences
0,Emma,20818
1,Evelyn,23283
2,Elizabeth,23125
3,Eleanor,14832
4,Grace,12741
5,Hazel,12765


In [7]:

-- most common baby names for each gender in each year
WITH female_names AS (
    SELECT 
        year,
        first_name,
        SUM(num) AS total_records,
        RANK() OVER (PARTITION BY year ORDER BY SUM(num) DESC) AS rnk
    FROM baby_names
    WHERE sex = 'F'
    GROUP BY year, first_name
),
male_names AS (
    SELECT 
        year,
        first_name,
        SUM(num) AS total_records,
        RANK() OVER (PARTITION BY year ORDER BY SUM(num) DESC) AS rnk
    FROM baby_names
    WHERE sex = 'M'
    GROUP BY year, first_name
)

SELECT 
    f.year,
    f.first_name AS most_common_female,
    m.first_name AS most_common_male
FROM female_names AS f
JOIN male_names AS m ON f.year = m.year
WHERE f.rnk = 1 AND m.rnk = 1
ORDER BY f.year ASC;



,year,most_common_female,most_common_male
0,1920,Mary,John
1,1921,Mary,John
2,1922,Mary,John
3,1923,Mary,John
4,1924,Mary,Robert
...,...,...,...
96,2016,Emma,Noah
97,2017,Emma,Liam
98,2018,Emma,Liam
99,2019,Olivia,Liam
